In [1]:
import sys
import os
import time
import argparse
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from PIL import Image

import cv2
from skimage import io
import numpy as np
import craft_utils
import imgproc
import file_utils
import json
import zipfile

from craft import CRAFT

In [2]:
net = CRAFT()  # initialize
state_dict = torch.load("craft_mlt_25k.pth", map_location="cpu")

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k.replace("module.", "")  # remove 'module.' of dataparallel
    new_state_dict[name] = v

net.load_state_dict(new_state_dict)
net.eval()
None

In [3]:
image = imgproc.loadImage("data/0e2f315f1f068638932de997005ff9a6.jpg")
img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(
        image,
        1280,
        interpolation=cv2.INTER_LINEAR,
        mag_ratio=1.5,
    )
ratio_h = ratio_w = 1 / target_ratio

# preprocessing
x = imgproc.normalizeMeanVariance(img_resized)
x = torch.from_numpy(x).permute(2, 0, 1)  # [h, w, c] to [c, h, w]
x = Variable(x.unsqueeze(0))  # [c, h, w] to [b, c, h, w]

# forward pass
y, _ = net(x)

# make score and link map
# score_text = y[0, :, :, 0].cpu().data.numpy()
# score_link = y[0, :, :, 1].cpu().data.numpy()

# render results (optional)
# render_img = score_text.copy()
# render_img = np.hstack((render_img, score_link))
# ret_score_text = imgproc.cvt2HeatmapImg(render_img)

In [8]:
# np.max(render_img*255)
# render_img.shape
import torchvision.transforms as transforms

y[0, :, :, 0].unsqueeze(2).shape

# render_img = score_text.copy()
# render_img = np.expand_dims(render_img, 0)

# render_img = torch.Tensor(render_img)
# render_img = torch.clamp(render_img, 0, 1)
# render_img = render_img * (render_img > 0.01).float()
# render_img
# render_img.shape

# transform = transforms.ToPILImage()
# transform(render_img)

# ret_score_text = cv2.cvtColor(ret_score_text,cv2.COLOR_BGR2RGB)
# pil_im = Image.fromarray(ret_score_text)
# pil_im
# Image.fromarray(render_img * 255, mode='L')

torch.Size([544, 608, 1])